In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("data.csv", dtype = {"CRS_DEP_TIME":str, "DEP_TIME":str, "CRS_ARR_TIME":str, "ARR_TIME":str, "CANCELLED":bool, "DIVERTED":bool}, parse_dates=["FL_DATE"])

In [5]:
print(data.head())

   YEAR  QUARTER  MONTH  DAY_OF_MONTH  DAY_OF_WEEK    FL_DATE CARRIER ORIGIN  \
0  2015        1      1            23            5 2015-01-23      US    BUF   
1  2015        1      1            23            5 2015-01-23      US    CLT   
2  2015        1      1            23            5 2015-01-23      US    CLT   
3  2015        1      1            23            5 2015-01-23      US    LGA   
4  2015        1      1            23            5 2015-01-23      US    CLT   

  DEST CRS_DEP_TIME         ...          ARR_DELAY  ARR_DELAY_GROUP  \
0  CLT         0615         ...               -3.0             -1.0   
1  ALB         2220         ...               -7.0             -1.0   
2  BUF         0925         ...               -9.0             -1.0   
3  CLT         0630         ...              -19.0             -2.0   
4  BUF         2215         ...                7.0              0.0   

   CANCELLED DIVERTED DISTANCE  CARRIER_DELAY  WEATHER_DELAY  NAS_DELAY  \
0      False    F

In [6]:
import requests
from bs4 import BeautifulSoup